In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np


import tensorflow as tf


In [ ]:
df = pd.read_csv("/kaggle/input/final-dataset/finalized_extreme.csv")

In [ ]:
df.head(5)

In [ ]:
df.labels.value_counts()

In [ ]:
list(df.labels.value_counts())

In [ ]:
plt.figure(figsize=(10,10))

df.labels.value_counts().plot.pie(autopct="%0.0f%%")

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbe = LabelEncoder()
df['labels_encoded2'] = lbe.fit_transform(df['labels'])

In [ ]:
df.groupby('labels')['labels_encoded2'].value_counts()

In [ ]:
df["labels_encoded2"].value_counts()[df["labels_encoded2"].value_counts() <= 31]

In [ ]:
unknowns = [
    5, 36, 40, 9, 37, 27, 12, 31, 30, 39, 34, 32, 35, 19, 42
]
df.loc[df["labels_encoded2"].isin(unknowns), "labels"] = "UNK"

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbe = LabelEncoder()
df["labels_encoded2"] = lbe.fit_transform(df["labels"])

In [ ]:
labels_list = list(df.labels.unique())

In [ ]:
labels_list

In [ ]:
df.labels_encoded2.unique()

In [ ]:
df.labels.unique() , df.labels.nunique()

In [ ]:
!pip install datasets

In [ ]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification, TrainingArguments, Trainer

from datasets import Dataset

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import numpy as np

import pandas as pd



class Alberta:

    def __init__(self):

        self.tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')



    def dataset_setting(self, dataset_text_att, dataset_label_att):

        text = dataset_text_att.tolist()

        labels = dataset_label_att.tolist()

        data = {"text": text, "labels": labels}

        dataset = Dataset.from_dict(data)

        dataset = dataset.train_test_split(test_size=0.2)

        return dataset



    def tokenize_function(self, examples):

        return self.tokenizer(examples['text'], padding='max_length', truncation=True)



    def mapping(self, dataset):

        tokenized_dataset = dataset.map(self.tokenize_function, batched=True)

        return tokenized_dataset



    def setting_format(self, columns, tokenized_dataset):

        tokenized_dataset.set_format(type='torch', columns=columns)

        return tokenized_dataset



    def init_model(self, num_labels=2):

        model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=num_labels)

        return model



    def training_args(self, output_dir, evaluation_strategy, learning_rate,

                      per_device_train_batch_size, per_device_eval_batch_size,

                      num_train_epochs, weight_decay):

        training_args = TrainingArguments(

            output_dir=output_dir,

            evaluation_strategy=evaluation_strategy,

            learning_rate=learning_rate,

            per_device_train_batch_size=per_device_train_batch_size,

            per_device_eval_batch_size=per_device_eval_batch_size,

            num_train_epochs=num_train_epochs,

            weight_decay=weight_decay,

            save_strategy='epoch',

            save_steps=0

        )

        return training_args



    def compute_metrics(self, eval_pred):

        logits, labels = eval_pred

        predictions = np.argmax(logits, axis=1)

        accuracy = accuracy_score(labels, predictions)

        precision = precision_score(labels, predictions, average='weighted')

        recall = recall_score(labels, predictions, average='weighted')

        f1 = f1_score(labels, predictions, average='weighted')

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



    def trainer_setup(self, model, training_args, train_dataset, eval_dataset):

        trainer = Trainer(

            model=model,

            args=training_args,

            train_dataset=train_dataset,

            eval_dataset=eval_dataset,

            tokenizer=self.tokenizer,

            compute_metrics=self.compute_metrics  # Add compute_metrics function here

        )

        return trainer



    def train(self, trainer):

        print("Training has been initiated")

        trainer.train()

        print("Training Completed")



    def result(self, trainer):
        print(trainer.evaluate())


    def classification_report(self, trainer, eval_dataset):
        # Make predictions
        predictions = trainer.predict(eval_dataset)
        logits, labels = predictions.predictions, predictions.label_ids

        # Get the predicted classes
        preds = np.argmax(logits, axis=1)

        # Generate the classification report
        report = classification_report(labels, preds, target_names=['B-PARAMETER PROPN', 'B-COMPONENT NOUN', 'O NOUN', 'O AUX', 'O X',
        'O PUNCT', 'O VERB', 'O ADV', 'O ADJ', 'O ADP', 'O NUM',
        'B-PARAMETER NOUN', 'B-ACTION VERB', 'O CCONJ', 'O PRON',
        'B-STATE NOUN', 'B-FUNCTION VERB', 'B-ISSUE VERB', 'O PART',
        'B-COMPONENT ADJ', 'I-ACTION VERB', 'B-FUNCTION NOUN',
        'B-ACTION NOUN', 'O INTJ', 'B-ISSUE NOUN', 'I-STATE VERB',
        'O SCONJ', 'O PROPN', 'I-ACTION ADP', 'B-STATE PROPN',
        'B-ACTION PROPN', 'B-STATE ADJ', 'UNK', 'I-STATE NOUN',
        'B-ISSUE PROPN', 'B-PARAMETER ADJ', 'B-FUNCTION ADJ',
        'B-ISSUE ADJ', 'I-COMPONENT NOUN', 'B-STATE ADV',
        'B-COMPONENT VERB', 'I-FUNCTION VERB', 'B-STATE VERB',
        'I-ACTION NOUN', 'I-PARAMETER NOUN'])
        return report


In [ ]:
l = ['B-PARAMETER PROPN', 'B-COMPONENT NOUN', 'O NOUN', 'O AUX', 'O X',
        'O PUNCT', 'O VERB', 'O ADV', 'O ADJ', 'O ADP', 'O NUM',
        'B-PARAMETER NOUN', 'B-ACTION VERB', 'O CCONJ', 'O PRON',
        'B-STATE NOUN', 'B-FUNCTION VERB', 'B-ISSUE VERB', 'O PART',
        'B-COMPONENT ADJ', 'I-ACTION VERB', 'B-FUNCTION NOUN',
        'B-ACTION NOUN', 'O INTJ', 'B-ISSUE NOUN', 'I-STATE VERB',
        'O SCONJ', 'O PROPN', 'I-ACTION ADP', 'B-STATE PROPN',
        'B-ACTION PROPN', 'B-STATE ADJ', 'UNK', 'I-STATE NOUN',
        'B-ISSUE PROPN', 'B-PARAMETER ADJ', 'B-FUNCTION ADJ',
        'B-ISSUE ADJ', 'I-COMPONENT NOUN', 'B-STATE ADV',
        'B-COMPONENT VERB', 'I-FUNCTION VERB', 'B-STATE VERB',
        'I-ACTION NOUN', 'I-PARAMETER NOUN']
print(len(l))

In [ ]:
# df.message = df.message.astype(str)

In [ ]:
import wandb
wandb.login(key = "1d4c63cc7a44c2478cf37ccdf5c0661bdaa89c2c")

In [ ]:
# Dataset setup
alberta = Alberta()

dataset = alberta.dataset_setting(df['words'], df['labels_encoded2'])

tokenized_dataset = alberta.mapping(dataset)
# Tokenize each split separately
from imblearn.over_sampling import RandomOverSampler
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np

# Step 1: Extract Training Data
train_df = tokenized_dataset['train'].to_pandas()

# Step 2: Apply Oversampling
ros = RandomOverSampler(random_state=42, sampling_strategy = "minority")
x_resampled, y_resampled = ros.fit_resample(
    np.array(train_df['text']).reshape(-1, 1),
    train_df['labels']
)

# Convert to pandas DataFrame
oversampled_train_df = pd.DataFrame({
    "text": x_resampled.ravel(),
    "labels": y_resampled
})

# Step 3: Convert Back to Dataset
oversampled_train_dataset = Dataset.from_pandas(oversampled_train_df)

# Step 4: Replace Training Split in DatasetDict
tokenized_dataset = DatasetDict({
    "train": oversampled_train_dataset,
    "test": dataset['test']
})

# Step 5: Tokenize the Dataset
tokenized_dataset = tokenized_dataset.map(alberta.tokenize_function, batched=True)




# Explicitly select 'train' and 'test' splits for further processing

# Format tokenized datasets
formatted_train = alberta.setting_format(['input_ids', 'attention_mask', 'labels'], tokenized_dataset['train'])
formatted_test = alberta.setting_format(['input_ids', 'attention_mask', 'labels'], tokenized_dataset['test'])

# Initialize model
model = alberta.init_model(num_labels=45)
model.to("cuda")

# Define training arguments
training_args = alberta.training_args(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=10,
    weight_decay=0.02,
)

# Setup trainer
trainer = alberta.trainer_setup(
    model=model,
    training_args=training_args,
    train_dataset=formatted_train,
    eval_dataset=formatted_test
)

# Train the model
alberta.train(trainer)

# Evaluate and generate classification report
alberta.result(trainer)
report = alberta.classification_report(trainer, formatted_test)
print(report)